# Knowledge Structure Graph Generator

## 1. DB연결

In [1]:
import pymysql

In [2]:
juso_db = pymysql.connect(
    user='root', 
    passwd='1234', 
    host='127.0.0.1', 
    db='test', 
    charset='utf8'
)

In [3]:
cursor = juso_db.cursor(pymysql.cursors.DictCursor)

In [4]:
sql = "SELECT * FROM knowledge_structure;"
cursor.execute(sql)
result = cursor.fetchall()

In [5]:
import pandas as pd

result = pd.DataFrame(result)
print("------------------------------지식구조 목록 리스트------------------------------")
print(result) #지식구조 목록 리스트화

------------------------------지식구조 목록 리스트------------------------------
       DOC_ID COT_MODE                                KNOWLEDGE_STRUCTURE
0        hemp      scs  sourc,origin,1.0\nsourc,millennium,1.0\nsourc,...
1       kaist      scs  박사,과정,1.0\n공동,카이스트,1.0\n면역,세포,2.19\n기술,개발,2.26...
2          감초      scs  함량,재배,2.9\n효능,역할,3.21\n하루,이상,3.9\n이상,단맛,3.9\n한...
3          녹차      scs  순수,티백,1.52\n제품,제조,2.15\n효과,흥분,2.95\n제품,순수,2.95...
4          대마      scs  마약,법률,3.06\n약물,중독,3.26\n한국,미국,3.87\n필로폰,헤로인,4....
5         바이오      scs  소재,개발,3.07\n분야,성과,3.6\n기술,식품,3.71\n분야,식품,3.79\...
6    인공지능_이루다      scs  제공,특정,2.76\n대화,연인들,3.54\n개발사,여성,3.54\n말투,학습,4....
7   인공지능_이루다2      scs  특정,자사,1.63\n수집,변호사,2.1\n개인정보,유출,2.76\n개인정보,수집,...
8     인공지능_제조      scs  제조,역할,1.0\n산업,제조,1.34\n산업,역할,1.34\n혁신,인공지능,2.3...
9     인공지능_호텔      scs  호텔,서비스,1.99\n코로나,업계,2.76\n제공,정보,2.98\n제공,차량,2....
10          차      scs  남부,지방,2.71\n재배,지역,4.03\n식물,우려,4.81\n남부,지역,4.85...
11          콩      scs  두부,된장,1.8\n고기,짜파

In [6]:
knowledge_structure = result['KNOWLEDGE_STRUCTURE']

## 2. 데이터 선택

#### 작성할 파일 제목 정하기
* file_name = "파일명"

In [7]:
file_name = "바이오" 

#### 추출하기 원하는 인덱스 선정

In [8]:
condition = result['DOC_ID'] == file_name
result_content = result[condition]['KNOWLEDGE_STRUCTURE'].reset_index()

In [9]:
result_content['KNOWLEDGE_STRUCTURE'][0]

'소재,개발,3.07\n분야,성과,3.6\n기술,식품,3.71\n분야,식품,3.79\n기술,개발,3.8\n기술,우수,3.9\n인프라,특성,3.9\n지원,제품,3.98\n소재,생산,4.0\n소재,연구,4.14\n바이오,산업,4.25\n성과,특성,4.32\n바이오,시장,4.42\n연구개발,분야,4.43\n소재,제품,4.55\n제품,생산,4.55\n지원,기업,4.66\n기업,분야,4.66\n바이오,인프라,4.78\n지원,산업,4.78\n소재,정보,4.81\n산업,해수부,4.81\n정보,생산,4.81\n'

In [10]:
# data = knowledge_structure[1] #인덱스만 변경하기
data = result_content['KNOWLEDGE_STRUCTURE'][0]
print(data)

소재,개발,3.07
분야,성과,3.6
기술,식품,3.71
분야,식품,3.79
기술,개발,3.8
기술,우수,3.9
인프라,특성,3.9
지원,제품,3.98
소재,생산,4.0
소재,연구,4.14
바이오,산업,4.25
성과,특성,4.32
바이오,시장,4.42
연구개발,분야,4.43
소재,제품,4.55
제품,생산,4.55
지원,기업,4.66
기업,분야,4.66
바이오,인프라,4.78
지원,산업,4.78
소재,정보,4.81
산업,해수부,4.81
정보,생산,4.81



In [11]:
import re

contents = re.split('\n', data)
contents = contents[:-1] #맨 끝 데이터는 공백

In [12]:
#노드 추가
for content in contents:
    value = re.split(',', content) # ,단위로 구분하여 리스트화
    print(value)

['소재', '개발', '3.07']
['분야', '성과', '3.6']
['기술', '식품', '3.71']
['분야', '식품', '3.79']
['기술', '개발', '3.8']
['기술', '우수', '3.9']
['인프라', '특성', '3.9']
['지원', '제품', '3.98']
['소재', '생산', '4.0']
['소재', '연구', '4.14']
['바이오', '산업', '4.25']
['성과', '특성', '4.32']
['바이오', '시장', '4.42']
['연구개발', '분야', '4.43']
['소재', '제품', '4.55']
['제품', '생산', '4.55']
['지원', '기업', '4.66']
['기업', '분야', '4.66']
['바이오', '인프라', '4.78']
['지원', '산업', '4.78']
['소재', '정보', '4.81']
['산업', '해수부', '4.81']
['정보', '생산', '4.81']


## 3. Data JavaScript File 생성

### 파일 2개 생성
* keyword.js : data(nodes, edges)
* keyword.html : graph

### 01. keyword.js

In [13]:
import networkx as nx

In [14]:
G = nx.Graph()

In [15]:
# 에지리스트를 그래프에 넣기 : 그래프에 입력된 노드 확인용
for content in contents:
    tmp = re.split(',', content)
    G.add_edge(tmp[0], tmp[1], weight = tmp[2])

In [16]:
degree_list = [degree for degree in G.degree()] # 각 노드별 차수 저장

In [17]:
# 파일 저장용 nodes, edges
graph_nodes = "var nodes = "
graph_edges = "var edges = "

# 리스트 생성용
num = 0
nodes_list = []
reverse_graph = dict()

for name in G.nodes():
    # 딕셔너리 1개씩 생성하여 넣기
    tmp = dict()
    tmp['id'] = num
    tmp['label'] = name
    tmp['degree'] = degree_list[num][1]

    
    # 색상은 파랑색으로 지정
    tmp['color'] = "#97C2FC"
    
#     # 차수가 3이상이면 빨간색 지정
#     if(degree_list[num][1] >= 3):
#         tmp['color'] = "#FB7E81"
#     else:
#         tmp['color'] = "#97C2FC"
        
    nodes_list.append(tmp)   # {id: 0, label: 'hemp'}형태
    
    reverse_graph[name] = num   # {'hemp':0, 'cannabi':1}형태, 목적: 'hemp'검색시 바로 노드 번호 확인(편의성 목적)
    num += 1

In [18]:
nodes_list

[{'id': 0, 'label': '소재', 'degree': 5, 'color': '#97C2FC'},
 {'id': 1, 'label': '개발', 'degree': 2, 'color': '#97C2FC'},
 {'id': 2, 'label': '분야', 'degree': 4, 'color': '#97C2FC'},
 {'id': 3, 'label': '성과', 'degree': 2, 'color': '#97C2FC'},
 {'id': 4, 'label': '기술', 'degree': 3, 'color': '#97C2FC'},
 {'id': 5, 'label': '식품', 'degree': 2, 'color': '#97C2FC'},
 {'id': 6, 'label': '우수', 'degree': 1, 'color': '#97C2FC'},
 {'id': 7, 'label': '인프라', 'degree': 2, 'color': '#97C2FC'},
 {'id': 8, 'label': '특성', 'degree': 2, 'color': '#97C2FC'},
 {'id': 9, 'label': '지원', 'degree': 3, 'color': '#97C2FC'},
 {'id': 10, 'label': '제품', 'degree': 3, 'color': '#97C2FC'},
 {'id': 11, 'label': '생산', 'degree': 3, 'color': '#97C2FC'},
 {'id': 12, 'label': '연구', 'degree': 1, 'color': '#97C2FC'},
 {'id': 13, 'label': '바이오', 'degree': 3, 'color': '#97C2FC'},
 {'id': 14, 'label': '산업', 'degree': 3, 'color': '#97C2FC'},
 {'id': 15, 'label': '시장', 'degree': 1, 'color': '#97C2FC'},
 {'id': 16, 'label': '연구개발', 'de

In [19]:
graph_nodes += str(nodes_list) + ";"
print(graph_nodes)

var nodes = [{'id': 0, 'label': '소재', 'degree': 5, 'color': '#97C2FC'}, {'id': 1, 'label': '개발', 'degree': 2, 'color': '#97C2FC'}, {'id': 2, 'label': '분야', 'degree': 4, 'color': '#97C2FC'}, {'id': 3, 'label': '성과', 'degree': 2, 'color': '#97C2FC'}, {'id': 4, 'label': '기술', 'degree': 3, 'color': '#97C2FC'}, {'id': 5, 'label': '식품', 'degree': 2, 'color': '#97C2FC'}, {'id': 6, 'label': '우수', 'degree': 1, 'color': '#97C2FC'}, {'id': 7, 'label': '인프라', 'degree': 2, 'color': '#97C2FC'}, {'id': 8, 'label': '특성', 'degree': 2, 'color': '#97C2FC'}, {'id': 9, 'label': '지원', 'degree': 3, 'color': '#97C2FC'}, {'id': 10, 'label': '제품', 'degree': 3, 'color': '#97C2FC'}, {'id': 11, 'label': '생산', 'degree': 3, 'color': '#97C2FC'}, {'id': 12, 'label': '연구', 'degree': 1, 'color': '#97C2FC'}, {'id': 13, 'label': '바이오', 'degree': 3, 'color': '#97C2FC'}, {'id': 14, 'label': '산업', 'degree': 3, 'color': '#97C2FC'}, {'id': 15, 'label': '시장', 'degree': 1, 'color': '#97C2FC'}, {'id': 16, 'label': '연구개발', 'degree

In [20]:
print(reverse_graph) # graph_nodes를 뒤집어서 저장, 예를 들어  label명 : id명

{'소재': 0, '개발': 1, '분야': 2, '성과': 3, '기술': 4, '식품': 5, '우수': 6, '인프라': 7, '특성': 8, '지원': 9, '제품': 10, '생산': 11, '연구': 12, '바이오': 13, '산업': 14, '시장': 15, '연구개발': 16, '기업': 17, '정보': 18, '해수부': 19}


In [21]:
edges_list = []

edge_max = 0.0
edge_min = 7.0

for edge in G.edges(data='weight'):
#     print("step: " + edge[2])
    tmp = float(edge[2])

    #최댓값 저장
    if(edge_max <= tmp):
        edge_max = tmp
    
    #최솟값 저장
    if(edge_min >= tmp):
        edge_min = tmp
        
#     print("edge_max: " + str(edge_max))
#     print("edge_min: " + str(edge_min))
#     print()

In [22]:
# edges는 {'hemp', 'cannabi', '3.21'}형태
for edge in G.edges(data='weight'):
    tmp = dict()
    tmp['from']   = reverse_graph.get(edge[0])     # 'hemp'검색시 인덱스값
    tmp['to']     = reverse_graph.get(edge[1])     # 'cannabi'검색시 인덱스값
    tmp['label']  = edge[2]                        # 가중치 저장
    
    length = 7.0 - float(edge[2])
    convert_edge_min = 7.0 - edge_max
    minmax_scale = ((length - convert_edge_min) / (edge_max - edge_min)) * 500        # 길이를 min max 스케일링
    tmp['value']  = round(minmax_scale, 2)         # Edge 두께를 가중치의 크기에 따라 변경(숫자가 클수록 두껍게)
    
    length = float(edge[2])
    minmax_scale = ((length - edge_min)/(edge_max - edge_min)) * 500
    tmp['length'] = round(minmax_scale +220, 2)    # Edge 길이를 min max 스케일링
    
    tmp['color'] = "#d3d3d3"                      

    edges_list.append(tmp)

In [23]:
graph_edges += str(edges_list) + ";"
print(graph_edges)

var edges = [{'from': 0, 'to': 1, 'label': '3.07', 'value': 500.0, 'length': 220.0, 'color': '#d3d3d3'}, {'from': 0, 'to': 11, 'label': '4.0', 'value': 232.76, 'length': 487.24, 'color': '#d3d3d3'}, {'from': 0, 'to': 12, 'label': '4.14', 'value': 192.53, 'length': 527.47, 'color': '#d3d3d3'}, {'from': 0, 'to': 10, 'label': '4.55', 'value': 74.71, 'length': 645.29, 'color': '#d3d3d3'}, {'from': 0, 'to': 18, 'label': '4.81', 'value': 0.0, 'length': 720.0, 'color': '#d3d3d3'}, {'from': 1, 'to': 4, 'label': '3.8', 'value': 290.23, 'length': 429.77, 'color': '#d3d3d3'}, {'from': 2, 'to': 3, 'label': '3.6', 'value': 347.7, 'length': 372.3, 'color': '#d3d3d3'}, {'from': 2, 'to': 5, 'label': '3.79', 'value': 293.1, 'length': 426.9, 'color': '#d3d3d3'}, {'from': 2, 'to': 16, 'label': '4.43', 'value': 109.2, 'length': 610.8, 'color': '#d3d3d3'}, {'from': 2, 'to': 17, 'label': '4.66', 'value': 43.1, 'length': 676.9, 'color': '#d3d3d3'}, {'from': 3, 'to': 8, 'label': '4.32', 'value': 140.8, 'lengt

In [24]:
graph_data = graph_nodes + "\n\n" + graph_edges

In [25]:
print(graph_data)

var nodes = [{'id': 0, 'label': '소재', 'degree': 5, 'color': '#97C2FC'}, {'id': 1, 'label': '개발', 'degree': 2, 'color': '#97C2FC'}, {'id': 2, 'label': '분야', 'degree': 4, 'color': '#97C2FC'}, {'id': 3, 'label': '성과', 'degree': 2, 'color': '#97C2FC'}, {'id': 4, 'label': '기술', 'degree': 3, 'color': '#97C2FC'}, {'id': 5, 'label': '식품', 'degree': 2, 'color': '#97C2FC'}, {'id': 6, 'label': '우수', 'degree': 1, 'color': '#97C2FC'}, {'id': 7, 'label': '인프라', 'degree': 2, 'color': '#97C2FC'}, {'id': 8, 'label': '특성', 'degree': 2, 'color': '#97C2FC'}, {'id': 9, 'label': '지원', 'degree': 3, 'color': '#97C2FC'}, {'id': 10, 'label': '제품', 'degree': 3, 'color': '#97C2FC'}, {'id': 11, 'label': '생산', 'degree': 3, 'color': '#97C2FC'}, {'id': 12, 'label': '연구', 'degree': 1, 'color': '#97C2FC'}, {'id': 13, 'label': '바이오', 'degree': 3, 'color': '#97C2FC'}, {'id': 14, 'label': '산업', 'degree': 3, 'color': '#97C2FC'}, {'id': 15, 'label': '시장', 'degree': 1, 'color': '#97C2FC'}, {'id': 16, 'label': '연구개발', 'degree

In [26]:
f = open("./results/result_webgraph/data/" + file_name + ".js", 'w', encoding = 'utf8')
f.write(graph_data)
f.close()
print(file_name + ".js 파일 생성 완료")

바이오.js 파일 생성 완료


### 2. keyword.html

In [27]:
html_top = """
<!doctype html>
<html>
<head>
	<title>Knowledge Structure</title>
    """

In [28]:
vis_location = """<script type="text/javascript" src="../../lib/drawing/vis.js"></script>
    """

In [29]:
js_location = """<script src="../../results/result_webgraph/data/""" + file_name + """.js" charset="utf-8">
    </script>
    <style type="text/css">#mynetwork 
        {width: 100%;height: 1000px;border: 1px solid lightgray;}
    </style>
</head>
<body>
    """

In [30]:
title = """<p> Cheonsol of Knowledge Structure.</p>"""

In [31]:
body = """
    <div id="mynetwork"></div>
	<script type="text/javascript">
		var container = document.getElementById('mynetwork');
		var data = { nodes: nodes, edges: edges }; """

In [32]:
option = """
        var options = 
        { "nodes": {"shape" : "circle",
                    "font":{"size": 35}},
        "edges": {"scaling":{"max":70}, "font": {"size" : 40}, 
            "smooth": {"type":"continuous","roundness": 0.5}},
            "physics": {"barnesHut": {"centralGravity": 0.001},
            "minVelocity": 0.75}};
        """

In [33]:
bottom = """var network = new vis.Network(container, data, options);
	</script>
</body>
</html>
"""

In [34]:
html_text = html_top + vis_location + js_location + title + body + option + bottom

In [35]:
html_text

'\n<!doctype html>\n<html>\n<head>\n\t<title>Knowledge Structure</title>\n    <script type="text/javascript" src="../../lib/drawing/vis.js"></script>\n    <script src="../../results/result_webgraph/data/바이오.js" charset="utf-8">\n    </script>\n    <style type="text/css">#mynetwork \n        {width: 100%;height: 1000px;border: 1px solid lightgray;}\n    </style>\n</head>\n<body>\n    <p> Cheonsol of Knowledge Structure.</p>\n    <div id="mynetwork"></div>\n\t<script type="text/javascript">\n\t\tvar container = document.getElementById(\'mynetwork\');\n\t\tvar data = { nodes: nodes, edges: edges }; \n        var options = \n        { "nodes": {"shape" : "circle",\n                    "font":{"size": 35}},\n        "edges": {"scaling":{"max":70}, "font": {"size" : 40}, \n            "smooth": {"type":"continuous","roundness": 0.5}},\n            "physics": {"barnesHut": {"centralGravity": 0.001},\n            "minVelocity": 0.75}};\n        var network = new vis.Network(container, data, op

In [36]:
html_file = open('./results/result_webgraph/' + file_name + '.html', 'w')
html_file.write(html_text)
html_file.close()
print(file_name + ".html 파일 생성 완료")

바이오.html 파일 생성 완료
